In [1]:
!pip install pyCausalFS tabulate xgboost scipy feature_engine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.9/300.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 21.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import os
import math
from pyCausalFS.CBD.MBs.IAMB import IAMB
from pyCausalFS.CBD.MBs.BAMB import BAMB
from pyCausalFS.CBD.MBs.STMB import STMB
from pyCausalFS.CBD.MBs.MBOR import MBOR
from pyCausalFS.CBD.MBs.LCMB import LRH
from pyCausalFS.CBD.MBs.MMMB.MMMB import MMMB
from pyCausalFS.CBD.MBs.HITON.HITON_MB import HITON_MB
from pyCausalFS.CBD.MBs.HITON.HITON_PC import HITON_PC
from pyCausalFS.CBD.MBs.MMMB.MMPC import MMPC
from pyCausalFS.CBD.MBs.GSMB import GSMB
from pyCausalFS.CBD.MBs.fast_IAMB import fast_IAMB
from pyCausalFS.CBD.MBs.inter_IAMB import inter_IAMB
from pyCausalFS.CBD.MBs.IAMBnPC import IAMBnPC
from pyCausalFS.CBD.MBs.interIAMBnPC import interIAMBnPC
from pyCausalFS.CBD.MBs.FBEDk import FBED
from pyCausalFS.CBD.MBs.PCMB.PCMB import PCMB
from pyCausalFS.CBD.MBs.semi_HITON.semi_HITON_MB import semi_HITON_MB
from pyCausalFS.CBD.MBs.IPCMB.IPCMB import IPC_MB
from pyCausalFS.LSL.MBs.PCDbyPCD import PCDbyPCD
from pyCausalFS.LSL.MBs.MBbyMB import MBbyMB
from pyCausalFS.LSL.MBs.CMB.CMB import CMB
from pyCausalFS.CBD.MBs.KIAMB import KIAMB
from pyCausalFS.CBD.MBs.TIE_star.TIEs import TIE_p
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import ADASYN
from sklearn import model_selection
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, matthews_corrcoef as mcc_score,  roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from tabulate import tabulate
from scipy.io import arff
from sklearn import svm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, f_classif, RFECV, SequentialFeatureSelector
from feature_engine.selection import DropCorrelatedFeatures, SmartCorrelatedSelection

In [3]:
significance_val = 0.05

# Feature Selection Algorithms

In [4]:
features_list = {
    "camel-2.11.0.csv":  [0, 4, 5, 6, 10, 11, 12, 14, 15, 19, 20, 22, 34, 38, 39, 40, 44, 46, 47, 50, 53, 60, 62, 64], 
    "derby-10.5.1.1.csv" :  [0, 4, 5, 6, 10, 11, 14, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 52, 53, 58, 60, 62, 64],
    "activemq-5.3.0.csv" :  [0, 4, 5, 6, 10, 11, 12, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 53, 58, 59, 64], 
    "derby-10.3.1.4.csv" :  [0, 4, 5, 6, 10, 11, 14, 15, 19, 20, 22, 29, 37, 38, 39, 40, 41, 43, 44, 47, 52, 53, 57, 59, 60, 61, 64], 
    "activemq-5.2.0.csv" :  [0, 4, 5, 6, 10, 11, 12, 14, 15, 19, 20, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 53, 58, 59, 64], 
    "hive-0.12.0.csv" :  [0, 4, 5, 11, 14, 19, 20, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 50, 52, 53, 60, 61, 62, 63, 64], 
    "camel-2.10.0.csv" :  [0, 4, 5, 6, 10, 11, 12, 14, 19, 20, 22, 34, 38, 39, 40, 41, 44, 46, 47, 50, 53, 60, 62, 64], 
    "derby-10.2.1.6.csv" :  [0, 4, 5, 6, 10, 11, 14, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 52, 53, 57, 59, 61, 64], 
    "hbase-0.95.0.csv" :  [4, 5, 11, 14, 19, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 50, 52, 53, 57, 60, 61, 62, 63, 64], 
    "hive-0.10.0.csv" :  [0, 5, 11, 14, 19, 20, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 50, 52, 53, 60, 61, 62, 64], 
    "camel-2.9.0.csv" :  [0, 4, 5, 6, 10, 11, 12, 14, 19, 20, 22, 34, 38, 39, 40, 41, 44, 46, 47, 50, 53, 60, 61, 62, 64], 
    "camel-1.4.0.csv" :  [0, 4, 5, 6, 10, 11, 12, 14, 15, 19, 20, 22, 34, 38, 39, 40, 41, 44, 47, 53, 59, 60, 61, 63, 64],
    "activemq-5.1.0.csv" :  [0, 4, 5, 6, 10, 11, 12, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 53, 59, 60, 64], 
    "jruby-1.7.0.preview1.csv" :  [0, 4, 5, 6, 10, 11, 14, 15, 19, 20, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 50, 52, 53, 58, 60, 61, 62, 64], 
    "wicket-1.3.0-incubating-beta-1.csv" :  [0, 4, 5, 10, 11, 12, 14, 15, 19, 20, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 53, 58, 60, 61, 62, 64], 
    "hbase-0.95.2.csv" :  [4, 5, 6, 11, 14, 19, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 50, 52, 53, 57, 60, 61, 62, 63, 64], 
    "activemq-5.0.0.csv" : [0, 4, 5, 6, 10, 11, 12, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 53, 55, 59, 60, 61, 64], 
    "activemq-5.8.0.csv" : [0, 4, 5, 10, 11, 12, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 50, 58, 59, 60, 64], 
    "wicket-1.3.0-beta2.csv" :  [0, 4, 5, 10, 11, 12, 14, 15, 19, 20, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 53, 58, 60, 61, 62, 64], 
    "lucene-3.1.csv" :  [0, 4, 5, 6, 10, 14, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 52, 53, 60, 61, 62, 64], 
    "wicket-1.5.3.csv" :  [0, 4, 5, 10, 11, 12, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 50, 53, 59, 60, 61, 64],
    "prop-3.csv" :  [2, 5, 6, 7, 8, 9, 11, 12, 13, 14, 17, 19],
    "prop-4.csv" :  [2, 5, 6, 8, 9, 11, 12, 13, 14, 15, 17, 19],
    "prop-2.csv" :  [1, 2, 5, 6, 7, 8, 9, 11, 12, 13, 14, 17, 19], 
    "prop-1.csv" :  [2, 3, 5, 6, 8, 9, 11, 12, 13, 14, 16, 17, 19], 
    "prop-5.csv" :  [1, 2, 5, 6, 8, 9, 11, 12, 13, 14, 17, 18],
    "mylyn.csv": [1, 2, 3, 9, 11, 12, 13]   
}

# Data Processing

In [5]:
def get_list_of_csv(folder_path):
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    return csv_files

### AEEEM

In [6]:
def process_aeeem_dataset(df):
    df.columns = df.columns.str.strip()
    df.rename(columns={df.columns[-1]: 'temp'}, inplace=True)
    columns_to_drop = ['temp', 'classname', 'nonTrivialBugs', 'majorBugs', 'criticalBugs', 'highPriorityBugs']
    columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    df = df.drop(columns=columns_to_drop)
    df.loc[df['bugs'] > 0, 'bugs'] = 1
    return df

### JIRA

In [7]:
def process_jira_dataset(df):
    columns_to_drop = ['File', 'RealBug', 'HeuBug', 'HeuBugCount']
    df = df.drop(columns=columns_to_drop)
    df.loc[df['RealBugCount'] > 0, 'RealBugCount'] = 1
    return df

### TERA-PROMISE-ck

In [8]:
def process_promise_ck_dataset(df):
    columns_to_drop = ['Name', 'version']
    columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    df = df.drop(columns=columns_to_drop)  
    df.loc[df['bug'] > 0, 'bug'] = 1
    return df

# Models

In [9]:
models_list = {
#     "KNN": KNeighborsClassifier(),
#     "Logistic Regression": LogisticRegression(),
#     "Decision Tree": tree.DecisionTreeClassifier(),
#     "XgBoost": XGBClassifier(),
#     "Random Forest": RandomForestClassifier(),
    "SVM":  svm.SVC(kernel='rbf')
}
models_search_params = {
    "KNN": {  
        'n_neighbors' : [3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31],
        'weights' : ['uniform','distance'],
        'metric' : ['minkowski','euclidean','manhattan']
     },
    "Logistic Regression": {
        'penalty' : ['l1', 'l2', 'elasticnet'],
        'C' : [1000, 100, 10, 1, 0.1, 0.01, 0.001, 0.0001],
        'solver' : ['lbfgs','newton-cg','liblinear', 'newton-cholesky', 'sag', 'saga'],
        'max_iter' : [100, 1500, 3000]
     },
    "Decision Tree": {
        'max_features': [1, 2, 3, 5, 7, 10, 'log2','sqrt', None],
        'max_depth': [2, 3, 5, 7, 10, 20, 30, 40, 50, 60, 70, None],
        'min_samples_split': [1, 2, 3, 5, 7, 9, 10, 0.1, 0.2, 0.3],
        'min_samples_leaf': [1, 2, 3, 5, 7, 9, 10, 0.1, 0.2],
     },
    "Random Forest": {
        'max_depth': [2, 5, 10, None],
        'max_features': ['log2', 'sqrt', None],
    },
    "XgBoost": {
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 6, 10],
        'subsample': [0.5, 0.7, 1],
        'n_estimators': [100, 500]
    },
    "SVM": {
        'kernel': ['rbf']
    }
}

# METRICS

In [10]:
def g_measure_score(y_test, y_pred):
    TN, FP, FN, TP = confusion_matrix(y_test, y_pred).ravel()
    FPR = FP/(FP+TN)
    recall = recall_score(y_test, y_pred, average='macro')
    g_measure = (2*recall*(1-FPR))/(recall+(1-FPR))
    return g_measure

def bal_score(y_test, y_pred):
    TN, FP, FN, TP = confusion_matrix(y_test, y_pred).ravel()
    recall = recall_score(y_test, y_pred, average='macro')
    FPR = FP/(FP+TN)
    PF = FPR
    PD = recall
    bal = 1 - (math.sqrt((1-PD)*(1-PD)+(0-PF)*(0-PF))/math.sqrt(2))
    return bal

In [11]:
def get_best_model_params(X_train, y_train, model):
    classifier = models_list[model]
    cv = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    grid_obj = RandomizedSearchCV(classifier, models_search_params[model], n_iter = 25, cv = cv, scoring='roc_auc')
    grid_obj.fit(X_train, y_train)
    best_model_params = grid_obj.best_params_
    return best_model_params

In [12]:
def build_model(file_name, X, y, model, feat_algo):
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    mcc_scores = []
    roc_auc_scores = []
    g_measure_scores = []
    bal_scores = []

    best_model_params = {}
    MB = []
    
    kf = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    for train_index, test_index in kf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        adasyn = ADASYN()
        X_train, y_train = adasyn.fit_resample(X_train, y_train)
        
        if feat_algo != 'None' and len(MB)<1:
            balanced_df = pd.concat([pd.DataFrame(X_train), pd.Series(y_train, name='bug')], axis=1)
            MB = features_list[file_name]
            print(file_name, feat_algo, balanced_df.shape[1]-1, MB)
            
        if len(MB) >= 1 and feat_algo != 'None':
            X_train = X_train[:, MB]
            X_test = X_test[:, MB]
        elif feat_algo != 'None':
            return [file_name, feat_algo, '--', '--', '--', '--', '--', '--', '--']
        
#         if not best_model_params:
#             best_model_params = get_best_model_params(X_train, y_train, model)
#             print(file_name, feat_algo, model, best_model_params)
            
        classifier = models_list[model]
#         classifier.set_params(**best_model_params)
        
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        
        
        accuracy_scores.append(accuracy_score(y_test, y_pred))
        precision_scores.append(precision_score(y_test, y_pred, average='macro'))
        recall_scores.append(recall_score(y_test, y_pred, average='macro'))
        f1_scores.append(f1_score(y_test, y_pred, average='macro'))
        mcc_scores.append(mcc_score(y_test, y_pred))
        roc_auc_scores.append(roc_auc_score(y_test, y_pred))
        g_measure_scores.append(g_measure_score(y_test, y_pred))
        bal_scores.append(bal_score(y_test, y_pred))
    
    return [file_name, feat_algo, round(np.mean(accuracy_scores),2), round(np.mean(precision_scores), 2), round(np.mean(recall_scores), 2), round(np.mean(f1_scores), 2), round(np.mean(mcc_scores), 2), round(np.mean(roc_auc_scores),2), round(np.mean(g_measure_scores), 2), round(np.mean(bal_scores), 2)]

# TRAINING

### TERA-PROMISE-ck

In [13]:
folder_path = '/kaggle/input/defect-prediction/TeraPromise-defect-dataset/ck'
files_list = get_list_of_csv(folder_path)

for model in models_list.keys():
    print('----------------',model,'---------------------------')
    table = []
    table.append(["dataset", "feat_algo", "acc", "prec", "recall", "f1", "mcc", "roc_auc", "g-m", "bal"])
    for file_name in files_list:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)

        if df.shape[0]<=1500:
            continue
        
        df = process_promise_ck_dataset(df)
        X = df.drop(columns=[df.columns[-1]]).values
        y = df[df.columns[-1]].values
        

        table.append(build_model(file_name, X, y, model, 'auto-spear'))
        table.append(['--------', '--------', '--------', '--------', '--------', '--------', '--------', '--------', '--------', '--------'])
    print(tabulate(table))

---------------- SVM ---------------------------
prop-3.csv auto-spear 20 [2, 5, 6, 7, 8, 9, 11, 12, 13, 14, 17, 19]
prop-4.csv auto-spear 20 [2, 5, 6, 8, 9, 11, 12, 13, 14, 15, 17, 19]
prop-2.csv auto-spear 20 [1, 2, 5, 6, 7, 8, 9, 11, 12, 13, 14, 17, 19]
prop-1.csv auto-spear 20 [2, 3, 5, 6, 8, 9, 11, 12, 13, 14, 16, 17, 19]
prop-5.csv auto-spear 20 [1, 2, 5, 6, 8, 9, 11, 12, 13, 14, 17, 18]
----------  ----------  --------  --------  --------  --------  --------  --------  --------  --------
dataset     feat_algo   acc       prec      recall    f1        mcc       roc_auc   g-m       bal
prop-3.csv  auto-spear  0.65      0.56      0.63      0.53      0.18      0.63      0.64      0.64
--------    --------    --------  --------  --------  --------  --------  --------  --------  --------
prop-4.csv  auto-spear  0.75      0.56      0.63      0.56      0.18      0.63      0.7       0.7
--------    --------    --------  --------  --------  --------  --------  --------  --------  --------

### JIRA

In [14]:
folder_path = '/kaggle/input/defect-prediction/JIRA-defect-dataset'
files_list = get_list_of_csv(folder_path)

for model in models_list.keys():
    print('----------------',model,'---------------------------')
    table = []
    table.append(["dataset", "feat_algo", "acc", "prec", "recall", "f1", "mcc", "roc_auc", "g-m", "bal"])
    for file_name in files_list:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)

        if df.shape[0]<=1500:
            continue
        
        df = process_jira_dataset(df)
        X = df.drop(columns=[df.columns[-1]]).values
        y = df[df.columns[-1]].values
        
        table.append(build_model(file_name, X, y, model, 'auto-spear'))
        table.append(['--------', '--------', '--------', '--------', '--------', '--------', '--------', '--------', '--------', '--------'])
    print(tabulate(table))

---------------- SVM ---------------------------
camel-2.11.0.csv auto-spear 65 [0, 4, 5, 6, 10, 11, 12, 14, 15, 19, 20, 22, 34, 38, 39, 40, 44, 46, 47, 50, 53, 60, 62, 64]
derby-10.5.1.1.csv auto-spear 65 [0, 4, 5, 6, 10, 11, 14, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 52, 53, 58, 60, 62, 64]
activemq-5.3.0.csv auto-spear 65 [0, 4, 5, 6, 10, 11, 12, 15, 19, 20, 22, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 53, 58, 59, 64]
derby-10.3.1.4.csv auto-spear 65 [0, 4, 5, 6, 10, 11, 14, 15, 19, 20, 22, 29, 37, 38, 39, 40, 41, 43, 44, 47, 52, 53, 57, 59, 60, 61, 64]
activemq-5.2.0.csv auto-spear 65 [0, 4, 5, 6, 10, 11, 12, 14, 15, 19, 20, 29, 34, 37, 38, 39, 40, 41, 43, 44, 47, 53, 58, 59, 64]
hive-0.12.0.csv auto-spear 65 [0, 4, 5, 11, 14, 19, 20, 29, 34, 37, 38, 39, 40, 41, 43, 44, 46, 47, 50, 52, 53, 60, 61, 62, 63, 64]
camel-2.10.0.csv auto-spear 65 [0, 4, 5, 6, 10, 11, 12, 14, 19, 20, 22, 34, 38, 39, 40, 41, 44, 46, 47, 50, 53, 60, 62, 64]
derby-10.2.1.6.csv auto-spear 6

### AEEEM Defect Dataset

In [15]:
folder_path = '/kaggle/input/defect-prediction/AEEEM-defect-dataset'
files_list = get_list_of_csv(folder_path)

for model in models_list.keys():
    print('----------------',model,'---------------------------')
    table = []
    table.append(["dataset", "feat_algo", "acc", "prec", "recall", "f1", "mcc", "roc_auc", "g-m", "bal"])
    for file_name in files_list:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path, delimiter=';')
        
        if df.shape[0]<=1500:
            continue
            
        df = process_aeeem_dataset(df)
        X = df.drop(columns=[df.columns[-1]]).values
        y = df[df.columns[-1]].values
        
        table.append(build_model(file_name, X, y, model, 'auto-spear'))
        table.append(['--------', '--------', '--------', '--------', '--------', '--------', '--------', '--------', '--------', '--------'])
    print(tabulate(table))

---------------- SVM ---------------------------
mylyn.csv auto-spear 15 [1, 2, 3, 9, 11, 12, 13]
---------  ----------  --------  --------  --------  --------  --------  --------  --------  --------
dataset    feat_algo   acc       prec      recall    f1        mcc       roc_auc   g-m       bal
mylyn.csv  auto-spear  0.62      0.59      0.68      0.54      0.25      0.68      0.64      0.64
--------   --------    --------  --------  --------  --------  --------  --------  --------  --------
---------  ----------  --------  --------  --------  --------  --------  --------  --------  --------
